In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

In [5]:
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
class DesklibAIDetectionModel(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config):
        super().__init__(config)
        self.model = AutoModel.from_config(config)
        self.classifier = nn.Linear(config.hidden_size, 1)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        pooled_output = sum_embeddings / sum_mask

        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float())

        output = {"logits": logits}
        if loss is not None:
            output["loss"] = loss
        return output

def predict_single_text(text, model, tokenizer, device, max_len=768, threshold=0.5):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        probability = torch.sigmoid(logits).item()

    label = 1 if probability >= threshold else 0
    return probability, label

In [3]:
model_directory = "desklib/ai-text-detector-v1.01"
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = DesklibAIDetectionModel.from_pretrained(model_directory)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# --- Example Input text ---
text_ai = "AI detection refers to the process of identifying whether a given piece of content, such as text, images, or audio, has been generated by artificial intelligence. This is achieved using various machine learning techniques, including perplexity analysis, entropy measurements, linguistic pattern recognition, and neural network classifiers trained on human and AI-generated data. Advanced AI detection tools assess writing style, coherence, and statistical properties to determine the likelihood of AI involvement. These tools are widely used in academia, journalism, and content moderation to ensure originality, prevent misinformation, and maintain ethical standards. As AI-generated content becomes increasingly sophisticated, AI detection methods continue to evolve, integrating deep learning models and ensemble techniques for improved accuracy."
text_human = "It is estimated that a major part of the content in the internet will be generated by AI / LLMs by 2025. This leads to a lot of misinformation and credibility related issues. That is why if is important to have accurate tools to identify if a content is AI generated or human written"

# --- Run prediction ---
probability, predicted_label = predict_single_text(text_ai, model, tokenizer, device)
print(f"Probability of being AI generated: {probability:.4f}")
print(f"Predicted label: {'AI Generated' if predicted_label == 1 else 'Not AI Generated'}")

probability, predicted_label = predict_single_text(text_human, model, tokenizer, device)
print(f"Probability of being AI generated: {probability:.4f}")
print(f"Predicted label: {'AI Generated' if predicted_label == 1 else 'Not AI Generated'}")


Probability of being AI generated: 0.9974
Predicted label: AI Generated
Probability of being AI generated: 0.4245
Predicted label: Not AI Generated


In [ ]:
test_df = pd.read_csv('../Splits/combined/sub_test.csv')
# test_df = pd.read_csv('../Splits/harder_test/sub_test.csv')
test_df

In [13]:
def GenerateResult(source: str):
  probability, predicted_label = predict_single_text(source, model, tokenizer, device)
  return {"AI_percentage":probability,"Model_Answer":('AI' if predicted_label == 1 else 'human')}

In [14]:
GenerateResult(test_df['source'][0])

{'AI_percentage': 0.913817822933197, 'Model_Answer': 'AI'}

In [15]:
import pathlib
import httpx
import time
from IPython.display import clear_output
err_logs = []
answer = []

err_cnt = 0

for index, row in test_df[len(answer):].iterrows():
  while True:
    try:
      response = GenerateResult(row['source'])
      clear_output()
      print(f"ASKED : {index}")
      err_cnt = 0
      answer.append(response)
      break
    except Exception as e:
      err_cnt += 1
      err_logs.append((index,e))
      clear_output()
      print(response)
      print(f"ERROR at {index} : {e}")

ASKED : 2999


In [16]:
import re
import json


result_n = len(answer)
answer_df = test_df.iloc[:result_n]
answer_df['answer_dict'] = answer

<ipython-input-16-299eb7d63628>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_dict'] = answer


In [18]:
answer_df

,Unnamed: 0.1,writer,problem_id,submission_id,website,source,Unnamed: 0,answer_dict
0,0,AI,164A,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN,"{'AI_percentage': 0.913817822933197, 'Model_An..."
1,1,AI,NaN,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN,"{'AI_percentage': 0.9539291858673096, 'Model_A..."
2,2,AI,o61_may08_estate,NaN,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,1843.0,"{'AI_percentage': 0.9568642973899841, 'Model_A..."
3,3,human,NaN,13646458.0,https://codeforces.com,//Template\n\n// By Anudeep :)\n//Includes\n#i...,NaN,"{'AI_percentage': 0.6888331174850464, 'Model_A..."
4,4,AI,1032,NaN,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,998.0,"{'AI_percentage': 0.9636139273643494, 'Model_A..."
...,...,...,...,...,...,...,...,...
2995,2995,human,1000,306681.0,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,NaN,"{'AI_percentage': 0.9555792808532715, 'Model_A..."
2996,2996,human,1055,310229.0,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,NaN,"{'AI_percentage': 0.9458506107330322, 'Model_A..."
2997,2997,Human,NaN,NaN,https://firefly.gchan.moe,#include <bits/stdc++.h>\n\nusing namespace st...,NaN,"{'AI_percentage': 0.9090267419815063, 'Model_A..."
2998,2998,AI,888F,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN,"{'AI_percentage': 0.6803218126296997, 'Model_A..."


In [19]:
answer_df['AI_percentage'] = answer_df['answer_dict'].apply(lambda x: x['AI_percentage'])
answer_df['Model_Answer'] = answer_df['answer_dict'].apply(lambda x: x['Model_Answer'])

<ipython-input-19-b41db5947d5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['AI_percentage'] = answer_df['answer_dict'].apply(lambda x: x['AI_percentage'])
<ipython-input-19-b41db5947d5b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['Model_Answer'] = answer_df['answer_dict'].apply(lambda x: x['Model_Answer'])


In [20]:
answer_df['verdict'] = answer_df['writer'] == answer_df['Model_Answer']
answer_df = answer_df[['writer','Model_Answer','verdict','AI_percentage','website','source']]

<ipython-input-20-56e2672f4023>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['verdict'] = answer_df['writer'] == answer_df['Model_Answer']


In [22]:
filtered_df = pd.DataFrame()
filtered_df[['Label','Model_Answer','Verdict','AI-Percentage','Website']] = answer_df[['writer','Model_Answer','verdict','AI_percentage','website']]

In [23]:
filtered_df['Model_Answer'] = filtered_df['Model_Answer'].replace('Human', 'human')
filtered_df['Label'] = filtered_df['Label'].replace('Human', 'human')
filtered_df['Verdict'] = filtered_df['Label']==filtered_df['Model_Answer']
filtered_df = filtered_df[filtered_df['Model_Answer'].isin(['human', 'AI'])]
filtered_df

,Label,Model_Answer,Verdict,AI-Percentage,Website
0,AI,AI,True,0.913818,https://codeforces.com
1,AI,AI,True,0.953929,https://codeforces.com
2,AI,AI,True,0.956864,https://programming.in.th
3,human,AI,False,0.688833,https://codeforces.com
4,AI,AI,True,0.963614,https://programming.in.th
...,...,...,...,...,...
2995,human,AI,False,0.955579,https://programming.in.th
2996,human,AI,False,0.945851,https://programming.in.th
2997,human,AI,False,0.909027,https://firefly.gchan.moe
2998,AI,AI,True,0.680322,https://codeforces.com


In [ ]:
filtered_df.to_csv("./Results/_2/Result-Desklib_AI-Detector.csv",index=False)

In [25]:
accuracy = filtered_df['Verdict'].value_counts(normalize=True).get(True, 0)
print(f"AUC : {accuracy}")

AUC : 0.5126666666666667
